In [27]:
import pandas as pd
import re
import nltk
import numpy as np
import string
from collections import Counter
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [7]:
# lscs members
lscs = pd.read_csv(
    "lscs_members.csv",
    on_bad_lines="warn"
)

lscs.head()

/tmp/ipython-input-1788044632.py:2: ParserWarning: Skipping line 743: expected 13 fields, saw 15

  lscs = pd.read_csv(


,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/


In [9]:
# Define the houses
houses = {
    "Red": {"House Name": "House of Jasper", "Gem/Crystal": "Jasper", "Signature Color/Look": "earthy red/brown-red"},
    "Orange": {"House Name": "House of Carnelian", "Gem/Crystal": "Carnelian", "Signature Color/Look": "fiery orange"},
    "Yellow": {"House Name": "House of Citrine", "Gem/Crystal": "Citrine", "Signature Color/Look": "golden yellow"},
    "Green": {"House Name": "House of Aventurine", "Gem/Crystal": "Aventurine", "Signature Color/Look": "shimmering green"},
    "Light Blue": {"House Name": "House of Amazonite", "Gem/Crystal": "Amazonite", "Signature Color/Look": "turquoise blue-green"},
    "Indigo/Dark Blue": {"House Name": "House of Lapis Lazuli", "Gem/Crystal": "Lapis Lazuli", "Signature Color/Look": "deep royal blue"},
    "Purple": {"House Name": "House of Amethyst", "Gem/Crystal": "Amethyst", "Signature Color/Look": "violet to purple"},
    "White": {"House Name": "House of Moonstone", "Gem/Crystal": "Moonstone", "Signature Color/Look": "milky white with rainbow sheen"},
    "Black": {"House Name": "House of Obsidian", "Gem/Crystal": "Obsidian / Onyx", "Signature Color/Look": "jet black (volcanic glass)"},
    "Pink/Brown": {"House Name": "House of Rose Quartz", "Gem/Crystal": "Serpentine", "Signature Color/Look": "pink"}
}

# Convert to DataFrame
house_df = pd.DataFrame.from_dict(houses, orient="index").reset_index()

# Rename columns
house_df.rename(columns={"index": "Color Category"}, inplace=True)

# Add numeric ID column
house_df.insert(0, "House ID", range(len(house_df)))

# Save to CSV
house_df.to_csv("houses.csv", index=False)

house_df


,House ID,Color Category,House Name,Gem/Crystal,Signature Color/Look
0,0,Red,House of Jasper,Jasper,earthy red/brown-red
1,1,Orange,House of Carnelian,Carnelian,fiery orange
2,2,Yellow,House of Citrine,Citrine,golden yellow
3,3,Green,House of Aventurine,Aventurine,shimmering green
4,4,Light Blue,House of Amazonite,Amazonite,turquoise blue-green
5,5,Indigo/Dark Blue,House of Lapis Lazuli,Lapis Lazuli,deep royal blue
6,6,Purple,House of Amethyst,Amethyst,violet to purple
7,7,White,House of Moonstone,Moonstone,milky white with rainbow sheen
8,8,Black,House of Obsidian,Obsidian / Onyx,jet black (volcanic glass)
9,9,Pink/Brown,House of Rose Quartz,Serpentine,pink


In [11]:
# Clean your "interests" column
lscs["cleaned_interests"] = (
    lscs["interests"]
    .astype(str)
    .replace(r"[&/;\n|*+-]", ",", regex=True)  # added + and -
    .str.replace(r",+", ",", regex=True)        # collapse multiple commas
    .str.strip(", ")                            # strip leading/trailing commas/spaces
)

initial = lscs['cleaned_interests'].sort_values()
initial.to_csv("initial_keywords.csv")
get_nan = lscs[lscs['cleaned_interests'].str.contains('nan',case=False)]
initial

,cleaned_interests
746,1. Coding (I worked on a platform game with tw...
789,3D Modeling and Video Editing
5,?
708,"ACADEMIC,Math, Science, History, Philosophy,MU..."
777,"Anime, Deepwoken, Overwatch, Pokémon, Genshin ..."
...,...
37,what if ayaw ko
56,writing
802,"writing, drawing, tech, games, music"
456,"writing, graphic design, walking"


In [12]:
# Build phrase map automatically from dataframe
stop_words = set(stopwords.words('english'))

# Collect all words from the dataframe
all_words = []
for text in lscs['cleaned_interests']:
    tokens = [w.lower() for w in word_tokenize(str(text)) if w not in string.punctuation and w not in stop_words]
    all_words.extend(tokens)

# Detect top bigrams
bigram_finder = BigramCollocationFinder.from_words(all_words)
top_bigrams = bigram_finder.nbest(BigramAssocMeasures.likelihood_ratio, 20)  # top 20 bigrams

# Detect top trigrams
trigram_finder = TrigramCollocationFinder.from_words(all_words)
top_trigrams = trigram_finder.nbest(TrigramAssocMeasures.likelihood_ratio, 10)  # top 10 trigrams

# Create phrase map: join words with _ so they become single tokens
phrase_map = {" ".join(b): "_".join(b) for b in top_bigrams + top_trigrams}

# Define keyword extraction function with nouns and verbs
def extract_keywords_auto(sentence):
    sentence_lower = str(sentence).lower()

    # Apply phrase mapping
    for phrase, replacement in phrase_map.items():
        if phrase in sentence_lower:
            sentence_lower = sentence_lower.replace(phrase, replacement)

    # Tokenize
    words = word_tokenize(sentence_lower)
    # Remove punctuation and stopwords
    words = [w for w in words if w not in string.punctuation and w not in stop_words]

    # POS tagging
    tagged = pos_tag(words)

    # Keep nouns and verbs
    keywords = [word for word, tag in tagged if tag.startswith('NN') or tag.startswith('VB')]

    # Return unique keywords
    return list(set(keywords))

# Apply to DataFrame
lscs["keywords_list"] = lscs["cleaned_interests"].apply(extract_keywords_auto)

# Save and inspect
keywords = lscs['keywords_list'].sort_values()
keywords.to_csv('keywords.csv', index=False)

# Count empty keyword rows
empty_keywords = lscs['keywords_list'].apply(lambda x: len(x) == 0).sum()
print("Number of empty keyword rows: ", empty_keywords)

# Get rows with empty keywords
empty_rows = lscs[lscs['keywords_list'].apply(lambda x: len(x) == 0)]

# Display their names (or any column you want)
empty_names = empty_rows['full_name'].tolist()
print("Empty keywords found for these names:")
print(empty_names)


Number of empty keyword rows:  2
Empty keywords found for these names:
['Rejano, Hans Martin F.', 'Arous, Adam']


In [14]:
# Convert list of keywords into a comma-separated string
lscs["keywords"] = lscs["keywords_list"].apply(lambda x: ", ".join(x) if x else "")

# Sort the keywords column alphabetically
lscs_sorted = lscs.sort_values(by="keywords")

# Save the sorted CSV
lscs_sorted[['keywords']].to_csv("keywords_sorted.csv", index=False)

# If you want to save the original list column too
lscs_sorted['keywords_list'].to_csv("keywords_list_sorted.csv", index=False)

lscs_sorted

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,cleaned_interests,keywords_list,keywords
642,12505196,"Arous, Adam",NaN,adam_arous@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Information Technology,NaN,many,0976 216 3937,https://www.facebook.com/adam.arousii?mibextid...,many,[],
5,12107884,"Rejano, Hans Martin F.",NaN,hans_rejano@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science,NaN,?,0929 236 6554,fb.com/rejano03,?,[],
375,12411515,"Fernandez, Bullet Andre F.",NaN,bullet_fernandez@dlsu.edu.ph,NaN,AVP,CORPREL,CCS,BS - Computer Science Major in Software Techno...,NaN,"Programming, Leadership, Journalism, Outdoor A...",0960 340 9380,fb.me/bulletfern,"Programming, Leadership, Journalism, Outdoor A...","[activities, outdoor, programming, journalism,...","activities, outdoor, programming, journalism, ..."
195,12308994,"Valdez, Pulvert M.",NaN,pulvert_valdez@dlsu.edu.ph,NaN,VP,PUBLI,CCS,BS - CSST,NaN,magkacrush sa di ako gusto,09762893125,facebook.com/pulvert.valdez,magkacrush sa di ako gusto,"[ako, di, sa, magkacrush, gusto]","ako, di, sa, magkacrush, gusto"
357,12411035,"Agraviador, Nash Radley V.",NaN,nash_agraviador@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science major in Software Techno...,NaN,"Theatre, Production, Data Analysis",09088698753,https://www.facebook.com/share/1Yj9ekyYsu/,"Theatre, Production, Data Analysis","[analysis, data, production]","analysis, data, production"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,12506001,"Mayor, Elijah Rexson B.",NaN,elijah_mayor@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS Computer Science major in Network and Infor...,NaN,"Reading, writing, listening to music, biking, ...",0969 640 5538,https://www.facebook.com/elijahrexson.mayor,"Reading, writing, listening to music, biking, ...","[writing, reading, projects, coding, listening...","writing, reading, projects, coding, listening,..."
157,12307513,"Miranda, Juan Miguel B.",NaN,juan_miranda@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Studies Major in Software Techno...,NaN,"Games, anime, watching shows and youtube",0976 191 2939,https://www.facebook.com/jm.miranda.858063,"Games, anime, watching shows and youtube","[youtube, games, watching, shows]","youtube, games, watching, shows"
600,12504416,"Chan, Sean Martin Y.",NaN,sean_martin_y_chan@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSCS - Software Technology,NaN,Coding and learning New Programming languages\...,09228908582,https://www.facebook.com/sean.chan.714049,"Coding and learning New Programming languages,...","[youtube, playing, languages, netflix, program...","youtube, playing, languages, netflix, programm..."
156,12307327,"Llanes, Andre Gabriel",NaN,andre_llanes@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science Major in Software Techno...,NaN,"zenless zone zero, dogs",0956 029 6425,https://www.facebook.com/andregabgd/,"zenless zone zero, dogs","[zone, zenless, dogs]","zone, zenless, dogs"


In [18]:
# Only vectorize non-empty rows
non_empty = lscs["keywords"] != ""
vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(", "))
X = vectorizer.fit_transform(lscs.loc[non_empty, "keywords"])
print(X)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2759 stored elements and shape (851, 764)>
  Coords	Values
  (0, 451)	1.0
  (1, 265)	0.48305845537194503
  (1, 209)	0.8755881044724572
  (2, 263)	0.5521548488267525
  (2, 658)	0.8337415804175212
  (3, 451)	1.0
  (4, 716)	0.6038732722258111
  (4, 599)	0.6038732722258111
  (4, 45)	0.4077445502204509
  (4, 122)	0.32313855224671867
  (5, 248)	1.0
  (6, 451)	1.0
  (7, 45)	0.690491965609061
  (7, 19)	0.7233400620934356
  (8, 658)	0.5134141027184765
  (8, 282)	0.41397954020236793
  (8, 278)	0.43854166863162164
  (8, 124)	0.6104982426652928
  (9, 263)	0.31016140532967956
  (9, 674)	0.6175784199628811
  (9, 449)	0.5568970929913314
  (9, 61)	0.46071946524742585
  (10, 451)	1.0
  (11, 583)	1.0
  (12, 454)	0.44711437101339
  :	:
  (845, 265)	0.2253527708852278
  (845, 278)	0.3546728157635925
  (845, 503)	0.32107526146024723
  (845, 357)	0.4764238702831263
  (845, 330)	0.4399436606695584
  (845, 703)	0.5475434869962605
  (846, 45)	0.4272

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [21]:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X)

# Compute distances to cluster centers
distances = kmeans.transform(X)  # shape: (n_samples, n_clusters)

# Sort points by distance to each cluster center
sorted_idx = np.argsort(distances, axis=0)

# Assign points evenly to clusters
cluster_size = X.shape[0] // n_clusters
balanced_labels = np.zeros(X.shape[0], dtype=int)

assigned = np.zeros(X.shape[0], dtype=bool)
for cluster_id in range(n_clusters):
    count = 0
    for idx in sorted_idx[:, cluster_id]:
        if not assigned[idx] and count < cluster_size:
            balanced_labels[idx] = cluster_id
            assigned[idx] = True
            count += 1

# Assign remaining points (if X.shape[0] % n_clusters != 0)
for i, flag in enumerate(assigned):
    if not flag:
        balanced_labels[i] = np.random.randint(0, n_clusters)

# Save balanced labels into your DataFrame
lscs.loc[non_empty, "assigned_house"] = balanced_labels

# For rows outside `non_empty`, assign random clusters
lscs.loc[~non_empty, "assigned_house"] = np.random.randint(0, n_clusters, size=(~non_empty).sum())

# Ensure integer type
lscs["assigned_house"] = lscs["assigned_house"].astype(int)

lscs.to_csv('assigned_houses.csv', index=False)
lscs

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,cleaned_interests,keywords_list,keywords,assigned_house
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez,nan,[nan],nan,1
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/,"Gaming, Editing","[gaming, editing]","gaming, editing",3
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia,"Tech, games","[games, tech]","games, tech",4
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah,nan,[nan],nan,7
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/,"Basketball, Sketching, Coding, Volunteering","[volunteering, sketching, basketball, coding]","volunteering, sketching, basketball, coding",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,12546615,"Angeles, Immanuel Zion B.",NaN,Immanuel_angeles@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - CSST18,NaN,"Coding, working out, basketball, guitar, running",0921 588 4310,https://www.facebook.com/share/18nUrcwsUy/,"Coding, working out, basketball, guitar, running","[running, working, coding, basketball, guitar]","running, working, coding, basketball, guitar",3
849,12547891,"Bai, Wilfred Hans T.",NaN,wilfred_bai@dlsu.edu.ph,NaN,MEM,MEM,CLA,AB Psychology / BS Marketing Management,NaN,"Listening to music, playing badminton",0917 865 5565,https://www.facebook.com/wilfred.bai,"Listening to music, playing badminton","[playing, badminton, music, listening]","playing, badminton, music, listening",0
850,12548219,"Chua, Rayne Hyden S.",NaN,rayne_hyden_s_chua@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS - Computer Science in Software Technology,NaN,Playing games and watching shows.,0947 606 6403,https://www.facebook.com/rayne.chua.2024,Playing games and watching shows.,"[playing_games, watching, shows]","playing_games, watching, shows",2
851,12587613,"Soque, Michael Paulo, L",NaN,michael_soque@dlsu.edu.ph,NaN,MEM,MEM,CCS,Master in Information Security,NaN,NaN,0961 150 1515,https://www.facebook.com/BoyWonderOfTheFutureP...,nan,[nan],nan,7


In [22]:
# Create a mapping from House ID to House Name
house_id_to_name = dict(zip(house_df["House ID"], house_df["House Name"]))

# Replace assigned_house numbers with house names
lscs["assigned_house"] = lscs["assigned_house"].map(house_id_to_name)

# Save to CSV if needed
lscs.to_csv("assigned_houses.csv", index=False)

lscs.head()

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,cleaned_interests,keywords_list,keywords,assigned_house
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez,nan,[nan],nan,House of Carnelian
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/,"Gaming, Editing","[gaming, editing]","gaming, editing",House of Aventurine
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia,"Tech, games","[games, tech]","games, tech",House of Amazonite
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah,nan,[nan],nan,House of Moonstone
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/,"Basketball, Sketching, Coding, Volunteering","[volunteering, sketching, basketball, coding]","volunteering, sketching, basketball, coding",House of Amethyst


In [23]:
lscs['assigned_house'].value_counts().sort_index()

,count
assigned_house,
House of Amazonite,85
House of Amethyst,85
House of Aventurine,85
House of Carnelian,85
House of Citrine,85
House of Jasper,86
House of Lapis Lazuli,86
House of Moonstone,85
House of Obsidian,85


In [24]:
# Create a clean copy
official_lscs = lscs.copy()

# Drop unwanted columns
columns_to_drop = ["cleaned_interests", "keywords_list", "keywords"]
official_lscs = official_lscs.drop(columns=[col for col in columns_to_drop if col in official_lscs.columns])

# Save to CSV if needed
official_lscs = official_lscs.sort_values('assigned_house')
official_lscs.to_csv("official_lscs.csv", index=False)

official_lscs.head()

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,assigned_house
426,12413054,"Rabacal, Marc Arthurias",NaN,marc_rabacal@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSCS-CSE,NaN,Computers,0956 227 7216,https://www.facebook.com/share/19eqYQn27V/,House of Amazonite
99,12262740,"Donaire, Lorenzo C.",NaN,lorenzo_donaire@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSCS - Computer Science,NaN,"Gaming, music, saxophone, computers, tech",09171486261,https://www.facebook.com/share/19aQ1twrNp/?mib...,House of Amazonite
486,12414697,"Ortha, Gian Lorenzo, C.",NaN,gian_ortha@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science Major in Network Informa...,NaN,"Video games, Programming, Computers and genera...",0932 540 4010,https://www.facebook.com/gianlorenzoortha,House of Amazonite
492,12414891,Lazaro Hayden C,NaN,charlemagne_lazaro@dlsu.edu.ph,NaN,MEM,MEM,CCS,Bs-St,NaN,Gym and coding,09152876742,NaN,House of Amazonite
495,12414948,"Tan, Lance Jerwin S.",NaN,lance_jerwin_s_tan@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Science,BS - Computer Science Major in Software Techno...,NaN,games and sports!,09563692899,https://www.facebook.com/lance.tan.58555,House of Amazonite


In [26]:
top_n = 10  # number of top keywords per house
house_data = []

for house in sorted(lscs['assigned_house'].unique()):
    house_rows = lscs[lscs['assigned_house'] == house]
    all_keywords = sum(house_rows['keywords_list'], [])  # flatten all lists in the house
    keyword_counts = Counter(all_keywords)
    top_keywords = [kw for kw, _ in keyword_counts.most_common(top_n)]

    house_data.append({
        'House': house,
        'Count': len(house_rows),
        **{f'Top{i+1}': kw for i, kw in enumerate(top_keywords)}
    })

# Create DataFrame
df_house_summary = pd.DataFrame(house_data)

# Display sorted by House
print(df_house_summary)


                   House  Count         Top1         Top2        Top3  \
0     House of Amazonite     85  video_games  programming      gaming   
1      House of Amethyst     85        games          gym      guitar   
2    House of Aventurine     85       gaming  programming  basketball   
3     House of Carnelian     85          nan          NaN         NaN   
4       House of Citrine     85     watching       gaming     playing   
5        House of Jasper     86        music      reading      gaming   
6  House of Lapis Lazuli     86        music       gaming     reading   
7     House of Moonstone     85          nan       coding       games   
8      House of Obsidian     85          nan     computer        data   
9   House of Rose Quartz     86          nan  internships       tours   

                  Top4     Top5           Top6                 Top7  \
0               coding   sports      computers                music   
1          photography  playing        running        